In [29]:
!activate PythonGPU
import numpy as np
from scipy.stats import skewnorm, skew
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, accuracy_score

def simulate_data(classes, n_vars, n, max_mu, max_sigma, max_skew):
    #The multivariate skew normal number generator
    def rng(mu, sigma, skew, n=1):
        k = len(mu)
        if not (k == len(sigma) and k ==len(skew)): 
            raise Exception("Mu, Sigma and Skew should be same length")

        data = np.zeros((int(n),k))

        for i in range(k):
            data[:,i] = skewnorm.rvs(skew[i], loc=mu[i], scale=sigma[i], size=int(n)) 

        return data
    
    if(np.sum(classes) != 1):
        raise Exception("Classes dont sum up to 1")
        
    n_classes = len(classes)
#     sigma = np.random.randint(1,max_sigma,n_vars)
#     skew = np.random.randint(-max_skew,max_skew,n_vars)
#     mu =  np.random.randint(-max_mu, max_mu, (n_classes, n_vars))
    
    sigma = (max_sigma * np.random.rand(1,n_vars))[0]
    skew = ((2 * max_skew  * np.random.rand(1, n_vars)) - max_skew)[0]
    mu = (2 *  max_mu * np.random.rand(n_classes, n_vars)) - max_mu
    
    n_obs_class = np.round(np.dot(classes,n))
    
    data = np.zeros((int(np.sum(n_obs_class)),n_vars+1))
    for i in range(n_classes):
        #calculate indexes
        start = int(np.sum(n_obs_class[0:i]))
        end = int(np.sum(n_obs_class[0:i+1]))
        
        #set the data
        data[start:end,0] = i
        data[start:end,1:] = rng(mu[i,:], sigma, skew, n_obs_class[i])
        
    X = data[:,1:]
    y = data[:,0]
        
    X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.33, 
    random_state=42,
    stratify=y)
    
    return X_train, X_test, y_train, y_test

[1 1 1 1]
[-2 -2 -1 -2]
[[ 1  1 -2  0]
 [ 0 -2 -2 -1]
 [ 0  0 -1  0]
 [-1 -2 -1  1]
 [ 1  0  1  0]]
[1.91870459 1.95709201 0.67677567 1.37492433]
[-0.04756076 -1.35692566 -1.09711342 -0.67016279]
[[ 1.90329814  1.50542531  0.80261036 -0.57159806]
 [-0.24664611 -0.18440792  0.99091365  1.58875771]
 [ 0.20298406  1.42630122  0.85325438  0.15003145]
 [-0.92047723 -1.69937498 -0.17951784  0.53697878]
 [-1.77937368  0.11976502 -1.82261174 -0.92836946]]


ValueError: size does not match the broadcast shape of the parameters.

In [3]:
(2 *  3 * np.random.rand(1, 3)) - 2

array([[2.99871232, 1.05954745, 0.12373237]])

In [4]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def classify_lda(X_train, X_test, y_train, y_test, priors, plot=False):
    lda = LinearDiscriminantAnalysis(priors=priors)
    X_lda = lda.fit_transform(X_train, y_train)

    predictions = lda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("LDA Test accuracy "+ str(accuracy))
    print(predictions)

    if plot:    
        plt.xlabel('LD1')
        plt.ylabel('LD2')
        plt.scatter(
            X_lda[:,0],
            X_lda[:,1],
            c=y_train,
            cmap='Accent',
        )
        
    return {"method": "LDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": lda}

In [5]:
#Quadratic
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def classify_qda(X_train, X_test, y_train, y_test, priors):
    qda = QuadraticDiscriminantAnalysis(priors=priors)
    X_qda = qda.fit(X_train, y_train)



    predictions = qda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("QDA Test accuracy "+ str(accuracy))

    return {"method": "QDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": qda}

In [6]:
from sklearn.linear_model import LogisticRegression

def classify_logit(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(random_state=0, solver='lbfgs',
                             multi_class='multinomial').fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Logistic Test accuracy "+ str(accuracy))
    
    return {"method": "Logit", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": clf}

In [7]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

def classify_knn(X_train, X_test, y_train, y_test, n_neighbors):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='euclidean')
    knn.fit(X_train, y_train)

    predictions = knn.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("KNN-"+str(n_neighbors)+" Test accuracy "+ str(accuracy))
    
    return {"method": "KNN-"+str(n_neighbors), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": knn}

In [8]:
#Naive bayes
from sklearn.naive_bayes import GaussianNB

def classify_naivebayes(X_train, X_test, y_train, y_test, priors):
    NB = GaussianNB(priors)
    NB.fit(X_train, y_train)
    
    predictions = NB.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("Naive Bayes Test accuracy "+ str(accuracy))
    
    return {"method": "Naive Bayes", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": NB}

In [9]:
#SVM
from sklearn.svm import LinearSVC

def classify_svm(X_train, X_test, y_train, y_test):
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    
    predictions = svm.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("SVM Test accuracy "+ str(accuracy))
    
    return {"method": "SVM", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": svm}

In [10]:
from tensorflow.keras import layers
from tensorflow import keras

def classify_neuralnet(X_train, X_test, y_train, y_test, n_vars, n_classes, depth=1, nodes=10, epochs=20):
    inputs = keras.Input(shape=(n_vars,), name='obs')
    x = layers.Dense(nodes, activation='relu')(inputs)
    
    if(depth>1):
        for i in range(depth-1):
            x = layers.Dense(nodes, activation='relu')(x)
            
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='Dataset')

    display(model.summary())


    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=64,
                        epochs=epochs,
                        validation_split=0.2)

    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    print(predictions)

    accuracy = accuracy_score(y_test, predictions)
    print("Neural Network Test accuracy "+ str(accuracy))
    
    return {"method": "Net "+"-".join([str(nodes) for i in range(depth)])+ " E"+str(epochs), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": model}

In [30]:
from datetime import datetime
import pandas as pd
import time


np.random.seed(12345)

timelabel =  datetime.now().strftime("%H-%M-%S - %d-%m-%Y")


configs = [{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 2,
    "n": 500,
    "max_mu": 2,
    "max_sigma": 2,
    "max_skew": 1
},
# {
#     "classes": [0.25, 0.25, 0.25, 0.25],
#     "n_vars": 5,
#     "n": 10000,
#     "max_mu": 5,
#     "max_sigma": 5,
#     "max_skew": 5
# },
# {
#     "classes": [0.05, 0.05, 0.05, 0.85],
#     "n_vars": 5,
#     "n": 10000,
#     "max_mu": 5,
#     "max_sigma": 5,
#     "max_skew": 5
# },
# {
#     "classes": [0.25, 0.25, 0.25, 0.25],
#     "n_vars": 50,
#     "n": 10000,
#     "max_mu": 5,
#     "max_sigma": 5,
#     "max_skew": 5
# },
# {
#     "classes": [0.25, 0.25, 0.25, 0.25],
#     "n_vars": 5,
#     "n": 10000,
#     "max_mu": 2,
#     "max_sigma": 5,
#     "max_skew": 5
# },
] 

columns = ['method', 'accuracy','predictions', "model", "config"] + list(configs[0].keys())
results = pd.DataFrame(columns=columns)
results.style.format({
    'accuracy': '{:,.3%}'.format
})



#run analysis
for i, c in enumerate(configs):
    X_train, X_test, y_train, y_test = simulate_data(c["classes"], c["n_vars"], c["n"], c["max_mu"], c["max_sigma"], c["max_skew"])
    
    lda = classify_lda(X_train, X_test, y_train, y_test, c["classes"], False)                                     
    results = results.append({**lda, **c, "config":i+1},ignore_index=True)
    
    qda = classify_qda(X_train, X_test, y_train, y_test, c["classes"])                                     
    results = results.append({**qda, **c, "config":i+1},ignore_index=True)
    
    logit = classify_logit(X_train, X_test, y_train, y_test)                                     
    results = results.append({**logit, **c, "config":i+1},ignore_index=True)
    
    for k in [5,10,50,100]:
        knn = classify_knn(X_train, X_test, y_train, y_test, k)
        results = results.append({**knn, **c, "config":i+1},ignore_index=True)

    bayes = classify_naivebayes(X_train, X_test, y_train, y_test, c["classes"])
    results = results.append({**bayes, **c, "config":i+1},ignore_index=True)
    
    svm = classify_svm(X_train, X_test, y_train, y_test)
    results = results.append({**svm, **c, "config":i+1},ignore_index=True)
    
    for n in [{"d":1,"n":len(c["classes"]), "e":25}, {"d":4,"n":30, "e":10}, {"d":4,"n":50, "e":50}]:
        neuralnet = classify_neuralnet(X_train, X_test, y_train, y_test, c["n_vars"], len(c["classes"]),  depth=n["d"], nodes=n["n"], epochs=n["e"])                                 
        results = results.append({**neuralnet, **c, "config":i+1},ignore_index=True)
    
    print("Results after config "+str(i+1)+" of "+str(len(configs)))

    results.sort_values(by='accuracy', ascending=False, inplace=True)
        
    display(results.style.format({
    'accuracy': '{:,.3%}'.format
    }))
    

    #saving results to file
    results.drop(columns=['model']).to_pickle("./results/config "+str(i+1)+" of "+str(len(configs))+" "+timelabel+".pkl")

LDA Test accuracy 0.5575757575757576
[3. 0. 0. 1. 1. 3. 0. 1. 0. 3. 2. 1. 1. 0. 0. 3. 3. 1. 2. 3. 0. 2. 1. 2.
 1. 3. 3. 2. 0. 3. 3. 1. 2. 3. 1. 1. 3. 1. 3. 1. 3. 0. 2. 2. 0. 1. 3. 2.
 3. 1. 1. 3. 0. 3. 0. 3. 3. 2. 0. 1. 0. 0. 0. 1. 3. 0. 3. 3. 2. 0. 3. 1.
 1. 1. 2. 3. 3. 0. 0. 1. 1. 3. 3. 3. 0. 0. 2. 3. 2. 0. 2. 3. 1. 3. 3. 0.
 0. 3. 2. 3. 0. 0. 0. 3. 1. 2. 0. 0. 1. 2. 1. 3. 3. 0. 0. 1. 3. 1. 3. 1.
 0. 1. 3. 3. 0. 1. 3. 1. 1. 0. 3. 0. 3. 2. 3. 1. 1. 0. 3. 1. 1. 1. 0. 2.
 1. 0. 2. 1. 1. 1. 3. 1. 1. 3. 0. 3. 1. 0. 0. 3. 1. 0. 2. 2. 1.]
QDA Test accuracy 0.5272727272727272
Logistic Test accuracy 0.5636363636363636
KNN-5 Test accuracy 0.4484848484848485
KNN-10 Test accuracy 0.4909090909090909
KNN-50 Test accuracy 0.5393939393939394
KNN-100 Test accuracy 0.4727272727272727
Naive Bayes Test accuracy 0.5333333333333333
SVM Test accuracy 0.5636363636363636
Model: "Dataset"
_________________________________________________________________
Layer (type)                 Output Shape              P

C:\Users\Tim\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


None

Train on 268 samples, validate on 67 samples
Epoch 1/25
268/268 [==============================] - 1s 3ms/sample - loss: 1.4976 - accuracy: 0.1493 - val_loss: 1.4647 - val_accuracy: 0.1194
Epoch 2/25
268/268 [==============================] - 0s 217us/sample - loss: 1.4835 - accuracy: 0.1269 - val_loss: 1.4572 - val_accuracy: 0.1343
Epoch 3/25
268/268 [==============================] - 0s 202us/sample - loss: 1.4750 - accuracy: 0.1082 - val_loss: 1.4500 - val_accuracy: 0.1343
Epoch 4/25
268/268 [==============================] - 0s 216us/sample - loss: 1.4670 - accuracy: 0.1045 - val_loss: 1.4446 - val_accuracy: 0.1343
Epoch 5/25
268/268 [==============================] - 0s 224us/sample - loss: 1.4602 - accuracy: 0.1157 - val_loss: 1.4387 - val_accuracy: 0.1343
Epoch 6/25
268/268 [==============================] - 0s 194us/sample - loss: 1.4530 - accuracy: 0.1269 - val_loss: 1.4334 - val_accuracy: 0.1343
Epoch 7/25
268/268 [==============================] - 0s 190us/sample - loss: 1.4

None

Train on 268 samples, validate on 67 samples
Epoch 1/10
268/268 [==============================] - 2s 6ms/sample - loss: 1.3663 - accuracy: 0.2425 - val_loss: 1.3404 - val_accuracy: 0.2537
Epoch 2/10
268/268 [==============================] - 0s 276us/sample - loss: 1.3373 - accuracy: 0.3022 - val_loss: 1.3247 - val_accuracy: 0.3433
Epoch 3/10
268/268 [==============================] - 0s 313us/sample - loss: 1.3177 - accuracy: 0.4104 - val_loss: 1.3121 - val_accuracy: 0.4328
Epoch 4/10
268/268 [==============================] - 0s 340us/sample - loss: 1.2998 - accuracy: 0.4664 - val_loss: 1.2954 - val_accuracy: 0.4478
Epoch 5/10
268/268 [==============================] - 0s 265us/sample - loss: 1.2826 - accuracy: 0.4851 - val_loss: 1.2736 - val_accuracy: 0.4627
Epoch 6/10
268/268 [==============================] - 0s 287us/sample - loss: 1.2595 - accuracy: 0.4963 - val_loss: 1.2544 - val_accuracy: 0.4776
Epoch 7/10
268/268 [==============================] - 0s 369us/sample - loss: 1.2

None

Train on 268 samples, validate on 67 samples
Epoch 1/50
268/268 [==============================] - 2s 9ms/sample - loss: 1.3657 - accuracy: 0.3060 - val_loss: 1.3457 - val_accuracy: 0.3731
Epoch 2/50
268/268 [==============================] - 0s 463us/sample - loss: 1.3032 - accuracy: 0.4739 - val_loss: 1.2866 - val_accuracy: 0.4776
Epoch 3/50
268/268 [==============================] - 0s 246us/sample - loss: 1.2474 - accuracy: 0.4925 - val_loss: 1.2585 - val_accuracy: 0.4179
Epoch 4/50
268/268 [==============================] - 0s 231us/sample - loss: 1.2086 - accuracy: 0.4963 - val_loss: 1.2231 - val_accuracy: 0.4478
Epoch 5/50
268/268 [==============================] - 0s 209us/sample - loss: 1.1697 - accuracy: 0.4925 - val_loss: 1.1800 - val_accuracy: 0.4925
Epoch 6/50
268/268 [==============================] - 0s 358us/sample - loss: 1.1394 - accuracy: 0.5112 - val_loss: 1.1637 - val_accuracy: 0.5373
Epoch 7/50
268/268 [==============================] - 0s 325us/sample - loss: 1.1

,method,accuracy,predictions,model,config,classes,n_vars,n,max_mu,max_sigma,max_skew
2,Logit,56.364%,[3. 0. 0. 1. 1. 3. 0. 1. 0. 3. 2. 1. 1. 0. 0. 3. 3. 1. 2. 3. 0. 2. 1. 2. 1. 3. 3. 2. 0. 3. 3. 1. 1. 3. 1. 1. 3. 1. 3. 1. 3. 0. 1. 2. 0. 1. 3. 2. 3. 1. 1. 3. 0. 3. 0. 3. 3. 2. 0. 1. 0. 0. 0. 1. 3. 0. 3. 3. 0. 0. 3. 1. 1. 1. 2. 3. 3. 0. 0. 1. 1. 3. 3. 3. 0. 0. 2. 3. 2. 0. 2. 3. 1. 3. 3. 0. 0. 3. 2. 3. 0. 0. 0. 3. 1. 0. 0. 0. 1. 2. 1. 3. 3. 0. 0. 1. 3. 1. 3. 1. 0. 1. 3. 3. 0. 1. 3. 1. 1. 0. 3. 0. 3. 2. 3. 1. 1. 0. 3. 1. 1. 1. 0. 2. 1. 0. 2. 1. 1. 1. 3. 1. 1. 3. 0. 3. 1. 0. 0. 3. 1. 0. 2. 2. 1.],"LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='multinomial', n_jobs=None, penalty='l2', random_state=0, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)",1,"[0.25, 0.25, 0.25, 0.25]",2,500,2,2,1
8,SVM,56.364%,[3. 0. 0. 1. 1. 3. 0. 1. 0. 3. 3. 1. 1. 0. 0. 3. 3. 1. 0. 3. 0. 0. 1. 3. 1. 3. 3. 1. 0. 3. 3. 1. 1. 3. 1. 1. 3. 1. 3. 1. 3. 0. 1. 3. 0. 1. 3. 0. 3. 1. 1. 3. 0. 3. 0. 3. 3. 3. 0. 1. 0. 0. 0. 1. 3. 0. 3. 3. 0. 0. 3. 1. 1. 1. 1. 3. 3. 0. 0. 1. 1. 3. 3. 3. 0. 0. 2. 3. 0. 0. 1. 3. 1. 3. 3. 0. 0. 3. 3. 3. 0. 0. 0. 3. 1. 0. 0. 0. 1. 1. 1. 3. 3. 0. 0. 1. 3. 1. 3. 3. 0. 1. 3. 3. 0. 1. 3. 1. 1. 0. 3. 0. 3. 1. 3. 1. 1. 0. 3. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 3. 1. 1. 3. 0. 3. 1. 0. 0. 3. 1. 0. 1. 3. 1.],"LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, loss='squared_hinge', max_iter=1000, multi_class='ovr', penalty='l2', random_state=None, tol=0.0001, verbose=0)",1,"[0.25, 0.25, 0.25, 0.25]",2,500,2,2,1
0,LDA,55.758%,[3. 0. 0. 1. 1. 3. 0. 1. 0. 3. 2. 1. 1. 0. 0. 3. 3. 1. 2. 3. 0. 2. 1. 2. 1. 3. 3. 2. 0. 3. 3. 1. 2. 3. 1. 1. 3. 1. 3. 1. 3. 0. 2. 2. 0. 1. 3. 2. 3. 1. 1. 3. 0. 3. 0. 3. 3. 2. 0. 1. 0. 0. 0. 1. 3. 0. 3. 3. 2. 0. 3. 1. 1. 1. 2. 3. 3. 0. 0. 1. 1. 3. 3. 3. 0. 0. 2. 3. 2. 0. 2. 3. 1. 3. 3. 0. 0. 3. 2. 3. 0. 0. 0. 3. 1. 2. 0. 0. 1. 2. 1. 3. 3. 0. 0. 1. 3. 1. 3. 1. 0. 1. 3. 3. 0. 1. 3. 1. 1. 0. 3. 0. 3. 2. 3. 1. 1. 0. 3. 1. 1. 1. 0. 2. 1. 0. 2. 1. 1. 1. 3. 1. 1. 3. 0. 3. 1. 0. 0. 3. 1. 0. 2. 2. 1.],"LinearDiscriminantAnalysis(n_components=None, priors=[0.25, 0.25, 0.25, 0.25], shrinkage=None, solver='svd', store_covariance=False, tol=0.0001)",1,"[0.25, 0.25, 0.25, 0.25]",2,500,2,2,1
10,Net 30-30-30-30 E10,55.152%,[3 2 0 1 1 3 0 1 0 3 3 1 1 0 0 3 3 1 1 1 0 0 1 3 1 3 3 1 0 3 3 1 1 3 1 1 3 1 3 1 3 0 1 3 0 1 3 0 3 1 1 3 0 1 0 3 3 3 0 1 0 0 0 1 3 0 3 3 0 0 0 1 1 1 1 3 3 0 0 1 1 3 1 3 0 0 3 3 0 0 1 0 1 3 3 0 0 3 3 3 0 0 0 3 1 0 0 0 1 1 1 3 3 0 0 1 3 1 3 1 0 1 3 3 1 1 3 1 1 0 3 0 3 1 3 1 1 0 3 1 1 1 0 3 1 0 0 1 1 1 3 1 1 3 0 1 1 0 0 3 1 0 1 3 1],,1,"[0.25, 0.25, 0.25, 0.25]",2,500,2,2,1
5,KNN-50,53.939%,[3. 2. 0. 1. 1. 0. 0. 1. 0. 2. 2. 2. 1. 0. 0. 1. 3. 1. 2. 1. 0. 0. 2. 2. 1. 2. 3. 2. 0. 3. 1. 1. 2. 3. 1. 1. 3. 2. 3. 1. 3. 0. 2. 2. 0. 1. 3. 2. 2. 1. 1. 3. 0. 1. 0. 3. 3. 2. 0. 1. 0. 0. 0. 1. 2. 0. 3. 3. 0. 0. 0. 1. 1. 1. 2. 3. 3. 0. 0. 1. 1. 3. 1. 3. 0. 0. 2. 2. 2. 0. 2. 0. 2. 3. 3. 0. 0. 3. 2. 2. 0. 0. 0. 3. 1. 0. 0. 0. 1. 2. 1. 3. 3. 0. 0. 2. 3. 1. 3. 2. 0. 1. 3. 1. 2. 1. 3. 1. 1. 0. 3. 0. 3. 2. 2. 1. 1. 0. 2. 1. 2. 2. 0. 2. 1. 0. 0. 1. 1. 1. 3. 2. 2. 3. 0. 1. 1. 0. 0. 3. 1. 0. 2. 2. 1.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=50, p=2, weights='uniform')",1,"[0.25, 0.25, 0.25, 0.25]",2,500,2,2,1
11,Net 50-50-50-50 E50,53.939%,[3 2 0 2 1 3 3 1 0 3 2 2 1 2 0 3 3 1 2 3 0 2 1 2 1 3 3 2 0 3 3 2 2 3 1 1 3 2 3 1 3 0 2 2 0 1 3 2 2 1 1 3 2 3 0 3 3 2 0 2 3 0 0 1 3 0 3 3 3 2 3 3 1 1 2 3 3 0 2 1 1 3 3 3 0 0 2 2 2 0 2 3 2 3 3 0 0 3 2 3 0 0 0 3 1 2 0 0 2 2 1 3 3 2 0 2 3 1 3 3 0 2 3 3 2 1 3 1 1 0 3 0 3 2 3 1 1 0 3 1 1 2 0 2 1 0 2 1 1 1 3 1 2 3 0 3 1 2 0 3 1 0 2 2 1],,1,"[0.25, 0.25, 0.25, 0.25]",2,500,2,2,1
7,Naive Bayes,53.333%,[3. 0. 0. 2. 1. 2. 0. 3. 0. 3. 2. 1. 1. 2. 0. 3. 3. 1. 2. 3. 0. 2. 1. 2. 1. 3. 3. 2. 0. 3. 3. 1. 2. 3. 1. 1. 3. 2. 3. 1. 3. 0. 2. 2.

Example to load old results

In [36]:
import pandas as pd

old_results = pd.read_pickle("./results/config 1 of 1 20-19-32 - 02-12-2019.pkl")

display(old_results.style.format({
    'accuracy': '{:,.3%}'.format
    }))

,method,accuracy,predictions,config,classes,n_vars,n,max_mu,max_sigma,max_skew
2,Logit,56.364%,[3. 2. 0. 0. 2. 1. 1. 2. 0. 1. 3. 0. 1. 2. 2. 3. 3. 1. 2. 1. 3. 2. 2. 3. 3. 3. 3. 2. 2. 3. 2. 1. 2. 3. 1. 1. 3. 2. 3. 2. 1. 1. 3. 1. 2. 1. 0. 3. 3. 0. 3. 2. 0. 3. 1. 3. 3. 1. 0. 2. 0. 1. 0. 1. 1. 0. 2. 3. 1. 3. 1. 3. 0. 2. 2. 1. 2. 1. 1. 1. 2. 3. 1. 2. 2. 0. 1. 3. 0. 0. 2. 3. 1. 3. 2. 1. 1. 0. 1. 1. 3. 2. 0. 3. 2. 0. 0. 3. 0. 2. 2. 3. 2. 0. 2. 2. 3. 2. 2. 0. 0. 2. 3. 1. 1. 1. 1. 2. 1. 2. 2. 0. 2. 1. 3. 2. 2. 1. 2. 2. 2. 1. 0. 3. 1. 2. 0. 0. 1. 3. 3. 1. 0. 3. 0. 3. 3. 0. 3. 3. 2. 2. 1. 2. 2.],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
3,KNN-5,56.364%,[3. 2. 1. 0. 2. 1. 1. 2. 0. 2. 3. 0. 1. 2. 1. 3. 3. 1. 3. 1. 0. 2. 2. 1. 2. 3. 3. 2. 0. 3. 2. 3. 0. 3. 1. 1. 0. 0. 3. 3. 2. 1. 2. 1. 2. 2. 0. 3. 3. 3. 2. 2. 0. 3. 0. 3. 3. 1. 0. 2. 0. 1. 0. 1. 1. 0. 2. 3. 2. 3. 2. 3. 2. 2. 2. 1. 2. 2. 1. 1. 2. 3. 2. 2. 2. 0. 2. 3. 0. 0. 2. 0. 1. 1. 2. 1. 1. 0. 1. 2. 3. 2. 0. 3. 0. 0. 0. 3. 0. 2. 1. 3. 2. 0. 2. 2. 2. 2. 0. 3. 3. 2. 3. 2. 1. 1. 3. 2. 1. 2. 2. 0. 2. 0. 3. 3. 2. 1. 2. 0. 2. 1. 0. 3. 1. 2. 0. 0. 2. 3. 2. 1. 1. 1. 0. 3. 3. 0. 0. 3. 1. 0. 3. 2. 2.],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
5,KNN-50,56.364%,[3. 2. 0. 0. 2. 3. 2. 2. 3. 1. 3. 0. 1. 2. 2. 3. 3. 1. 2. 1. 3. 2. 2. 3. 3. 3. 0. 2. 1. 3. 2. 1. 2. 3. 1. 1. 0. 0. 3. 2. 1. 1. 3. 1. 1. 2. 0. 3. 3. 0. 3. 2. 0. 2. 1. 3. 3. 1. 0. 2. 0. 1. 0. 1. 1. 0. 2. 3. 1. 0. 2. 3. 0. 2. 2. 1. 2. 1. 1. 1. 2. 3. 1. 2. 2. 0. 1. 1. 0. 0. 2. 0. 1. 3. 2. 1. 1. 0. 1. 2. 3. 2. 0. 3. 2. 0. 0. 3. 2. 2. 1. 3. 2. 0. 2. 2. 3. 2. 2. 0. 0. 2. 3. 1. 1. 2. 1. 2. 1. 2. 2. 0. 2. 1. 3. 2. 2. 1. 2. 2. 2. 1. 0. 3. 1. 2. 0. 1. 1. 3. 3. 1. 0. 3. 0. 3. 2. 0. 3. 3. 2. 2. 1. 2. 2.],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
8,SVM,56.364%,[3. 2. 1. 0. 2. 1. 1. 2. 0. 1. 3. 0. 1. 2. 2. 3. 3. 1. 2. 1. 3. 2. 2. 3. 3. 3. 3. 2. 2. 3. 2. 1. 2. 3. 1. 1. 0. 2. 3. 2. 1. 1. 3. 1. 1. 1. 0. 3. 3. 0. 3. 2. 0. 3. 1. 3. 3. 1. 0. 2. 0. 1. 0. 1. 1. 0. 2. 3. 1. 0. 1. 3. 0. 1. 2. 1. 2. 1. 1. 1. 2. 3. 1. 2. 1. 0. 1. 3. 0. 0. 2. 3. 1. 3. 2. 1. 1. 0. 1. 1. 3. 3. 0. 3. 2. 0. 0. 3. 0. 2. 2. 3. 2. 0. 2. 2. 3. 2. 2. 0. 3. 2. 3. 1. 1. 1. 1. 2. 1. 2. 2. 0. 2. 1. 3. 2. 2. 1. 2. 2. 2. 1. 0. 3. 1. 2. 0. 0. 1. 3. 3. 1. 0. 3. 0. 3. 3. 0. 3. 3. 2. 2. 1. 2. 2.],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
6,KNN-100,55.758%,[3. 2. 1. 0. 2. 1. 2. 2. 0. 1. 3. 0. 1. 2. 2. 2. 3. 1. 2. 1. 3. 2. 2. 3. 3. 3. 0. 2. 1. 3. 2. 1. 2. 3. 1. 1. 0. 0. 3. 2. 2. 1. 3. 1. 1. 1. 0. 3. 3. 0. 3. 2. 0. 2. 1. 3. 3. 1. 0. 2. 0. 1. 0. 1. 1. 0. 2. 3. 1. 0. 2. 3. 0. 1. 2. 1. 2. 1. 1. 1. 2. 3. 1. 2. 1. 0. 1. 3. 0. 0. 2. 3. 1. 3. 2. 1. 1. 0. 1. 1. 3. 2. 0. 3. 2. 0. 0. 3. 0. 2. 1. 3. 2. 1. 2. 2. 3. 2. 2. 0. 1. 2. 3. 1. 1. 1. 3. 2. 1. 2. 2. 0. 2. 1. 3. 2. 2. 1. 2. 2. 2. 1. 0. 3. 1. 2. 0. 1. 1. 3. 3. 1. 0. 3. 0. 3. 3. 0. 3. 3. 2. 2. 3. 2. 2.],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
11,Net 30-30-30-30 E50,55.758%,[3 2 1 0 2 3 2 2 1 1 3 0 1 2 2 2 3 1 2 1 3 2 2 3 2 3 3 2 1 3 2 1 2 1 1 1 0 2 3 2 2 3 3 1 1 1 0 3 3 3 3 2 0 3 1 3 3 1 0 2 0 1 0 1 3 0 2 3 1 3 2 3 0 2 2 2 2 1 1 1 2 3 2 2 2 0 2 1 0 0 2 3 1 3 2 0 1 0 1 2 3 2 0 3 2 0 0 3 2 2 1 3 2 0 2 2 3 2 2 0 0 2 3 2 1 3 3 2 1 2 2 0 2 1 3 3 2 1 2 1 2 1 0 1 1 2 0 1 1 3 3 1 1 3 0 3 3 3 3 3 2 2 1 2 2],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
0,LDA,55.152%,[3. 2. 1. 0. 2. 1. 1. 2. 0. 1. 3. 0. 1. 2. 2. 2. 3. 1. 2. 1. 3. 2. 2. 3. 2. 3. 0. 2. 2. 3. 2. 1. 2. 3. 1. 1. 0. 2. 3. 2. 1. 1. 3. 1. 1. 1. 0. 3. 3. 0. 3. 2. 0. 3. 1. 3. 3. 1. 0. 2. 0. 1. 0. 1. 1. 0. 2. 3. 1. 3. 1. 3. 0. 1. 2. 1. 2. 1. 1. 1. 2. 3. 1. 2. 2. 0. 1. 3. 0. 0. 2. 3. 1. 3. 2. 1. 1. 0. 1. 1. 3. 3. 0. 3. 2. 0. 0. 3. 0. 2. 2. 3. 2. 0. 2. 2. 3. 2. 2. 0. 0. 2. 3. 1. 1. 3. 1. 2. 1. 2. 2. 0. 2. 1. 3. 2. 2. 1. 2. 2. 2. 1. 0. 3. 1. 2. 0. 0. 1. 3. 3. 1. 1. 3. 0. 3. 3. 0. 3. 3. 2. 2. 1. 2. 2.],1,"[0.25, 0.25, 0.25, 0.25]",5,500,1,2,1
1,QDA,55.152%,[3. 2. 0. 0. 2. 1. 3. 2. 3. 1. 3. 0. 1. 2. 2. 3. 3. 1. 2. 1. 3. 2. 2. 3. 3. 3. 0. 3. 1. 3. 2. 1. 2. 1. 1. 1. 0. 0. 3. 3. 2. 1. 3. 1. 1. 1. 0. 3. 3. 3. 3. 2. 0. 3. 1. 3. 3. 1. 0. 2.